In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
import pandas as pd
def replace_nan_with_average(series):
    # Identify the indices of NaN values
    nan_indices = np.where(np.isnan(series))[0]
    
    for idx in nan_indices:
        print("yes")
        # Ensure there are previous and next values to average
        if idx > 0 and idx < len(series) - 1:
            previous_value = series[idx - 1]
            next_value = series[idx + 1]
            if not np.isnan(previous_value) and not np.isnan(next_value):
                series[idx] = (previous_value + next_value) / 2.0
            elif not np.isnan(previous_value):
                series[idx] = previous_value
            elif not np.isnan(next_value):
                series[idx] = next_value
    return series
def model (data,time, α1,α2,α3,β11,β12,β13,β21,β22,β23,β31,β32,β33,γ1,γ2,γ3,A_b1,A_b2,A_b3,R_b1,R_b2,R_b3): 
    A1,R1,V1,A2,R2,V2,A3,R3,V3=data
    dA1dt=-1*(α1+γ1*V1)*(A1-A_b1)+(β11*(A1+V1)+β21*(A2+V2)+β31*(A3+V3))*(R1-R_b1)
    dR1dt=-(β11*(A1+V1)+β21*(A2+V2)+β31*(A3+V3))*(R1-R_b1)
    dV1dt=(α1+γ1*V1)*(A1-A_b1)
    dA2dt=-1*(α2+γ2*V2)*(A2-A_b2)+(β12*(A1+V1)+β22*(A2+V2)+β32*(A3+V3))*(R2-R_b2)
    dR2dt=-(β12*(A1+V1)+β22*(A2+V2)+β32*(A3+V3))*(R2-R_b2)
    dV2dt=(α2+γ2*V2)*(A2-A_b2)
    dA3dt=-1*(α3+γ3*V3)*(A3-A_b3)+(β13*(A1+V1)+β23*(A2+V2)+β33*(A3+V3))*(R3-R_b3)
    dR3dt=-(β13*(A1+V1)+β23*(A2+V2)+β33*(A3+V3))*(R3-R_b3)
    dV3dt=(α3+γ3*V3)*(A3-A_b3)
    return [dA1dt,dR1dt,dV1dt,dA2dt,dR2dt,dV2dt,dA3dt,dR3dt,dV3dt]
def loss(parameters, data_0, time, data): 
    α1,α2,α3,β11,β12,β13,β21,β22,β23,β31,β32,β33,γ1,γ2,γ3,A_b1,A_b2,A_b3,R_b1,R_b2,R_b3= parameters
    data_predict = odeint(model, data_0, time, args=(α1,α2,α3,β11,β12,β13,β21,β22,β23,β31,β32,β33,γ1,γ2,γ3,A_b1,A_b2,A_b3,R_b1,R_b2,R_b3))
    A1_predict = data_predict[:, 0]
    R1_predict = data_predict[:, 1]
    V1_predict = data_predict[:, 2]
    A2_predict = data_predict[:, 3]
    R2_predict = data_predict[:, 4]
    V2_predict = data_predict[:, 5]
    A3_predict = data_predict[:, 6]
    R3_predict = data_predict[:, 7]
    V3_predict = data_predict[:, 8]
    return np.sum((A1_predict - data[0,:]) ** 2+(R1_predict - data[1,:]) ** 2+(V1_predict - data[2,:]) ** 2+(A2_predict - data[3,:]) ** 2+(R2_predict - data[4,:]) ** 2+(V2_predict - data[5,:]) ** 2+(A3_predict - data[6,:]) ** 2+(R3_predict - data[7,:]) ** 2+(V3_predict - data[8,:]) ** 2)
statelist=["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
    "New Hampshire", "New Jersey", "New Mexico", "New York",
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"]

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
import pandas as pd
df3=pd.read_csv("paras-agemod2.csv")
data_0=np.zeros([9])
xd=np.zeros([9,536])
i=0
s=0
zd=np.zeros([3,5])
statelist=df3["state"]
for state in statelist:
    if df3["politic"].iloc[i]=="R":
        print(state)
        df = pd.read_csv(f'age-weekly/{state}.csv')
        df2=df.drop(columns="Dates")
        dataa=df2.to_numpy().T
        xd+=dataa/np.sum(dataa,axis=0)
        s+=1
    i+=1
    df = pd.read_csv(f'age-weekly/'+state+'.csv')
    time=np.arange(0, len(df['Dates']), 1)
data_0=list(xd[:,0]/s)
xd=xd/s
for i in range(9):
    xd[i,:]=replace_nan_with_average(xd[i,:])
data_0=list(np.average(xd[:,0:8],axis=1))
Bounds=((0,1),(0,1),(0,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))
result = differential_evolution(loss, bounds=Bounds,args=(data_0, time,xd),strategy='best1bin',maxiter=1000, popsize=25,tol=1e-7,  atol=1e-7, mutation=(0.5, 1), recombination=0.7,   disp=False,polish=True )
best_params = result.x
print(best_params)
Analitical_data=(odeint(model, data_0, time, args=tuple(best_params))).T 
np.savetxt("age-Republic-model.txt",Analitical_data)
np.savetxt("age-Republic-survey.txt",xd)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Analitical_data=np.loadtxt("age-Republic-model.txt")
xd=np.loadtxt("age-Republic-model.txt")
mf=5
lf=2.7
titf=21
xlf=21
mf=5
lf=2.7
titf=21
xlf=21
ageg=["Age 18-34","Age 35-64", "Age above 64"]
fig=plt.figure(figsize=(10,7))
rect_1=[0,0,.7,.7]
ax1=fig.add_axes(rect_1,facecolor=(1,1,1))
ax1.plot(time,xd[0,:],".g",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[0,:],"--g",linewidth=lf,label="Accepted")
ax1.plot(time,xd[1,:],".r",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[1,:],"--r",linewidth=lf,label="Refused")
ax1.plot(time,xd[2,:],".b",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[2,:],"--b",linewidth=lf,label="Vaccinated")
plt.legend(fontsize=20,loc=2)
#plt.ylim(-0.01,0.5)
plt.xlim(0,546)
plt.xlabel(r"$Date$",fontsize=xlf+1)
labels=["Jan 2021","Apr 2021","Jul 2021","Oct 2021","Jan 2022","Apr 2022","Jul 2022"]
plt.xticks([0,90,181,272,365,455,546], labels,fontsize=13,rotation=45 )
#labels=["0.0","0.1","0.2","0.3","0.4","0.5"]
#plt.yticks([0,0.1,.2,0.3,.4,0.5], labels,fontsize=13 )
tit=ageg[0]
plt.title(tit,fontsize=titf)
plt.ylabel("Frequency",fontsize=xlf)
ax11=ax1.twinx()
ax11.plot(time,xd[0,:]/np.sum(xd[0:3,:],axis=0),".g",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[0,:]/np.sum(Analitical_data[0:3,:],axis=0),"--g",linewidth=lf)
ax11.plot(time,xd[1,:]/np.sum(xd[0:3,:],axis=0),".r",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[1,:]/np.sum(Analitical_data[0:3,:],axis=0),"--r",linewidth=lf)
ax11.plot(time,xd[2,:]/np.sum(xd[0:3,:],axis=0),".b",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[2,:]/np.sum(Analitical_data[0:3,:],axis=0),"--b",linewidth=lf)


rect_1=[0.9,0,.7,.7]
ax1=fig.add_axes(rect_1,facecolor=(1,1,1))
ax1.plot(time,xd[3,:],".g",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[3,:],"--g",linewidth=lf)
ax1.plot(time,xd[4,:],".r",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[4,:],"--r",linewidth=lf)
ax1.plot(time,xd[5,:],".b",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[5,:],"--b",linewidth=lf)

plt.xlim(0,546)
plt.xlabel(r"$Date$",fontsize=xlf+1)
labels=["Jan 2021","Apr 2021","Jul 2021","Oct 2021","Jan 2022","Apr 2022","Jul 2022"]
plt.xticks([0,90,181,272,365,455,546], labels,fontsize=13,rotation=45 )
tit=ageg[1]
plt.title(tit,fontsize=titf)
plt.ylabel("Frequency",fontsize=xlf)

#labels=[""]
#plt.yticks([0.0,0.2,0.4,0.6,0.8,1.0], labels,fontsize=13 )
ax11=ax1.twinx()
ax11.plot(time,xd[3,:]/np.sum(xd[3:6,:],axis=0),".g",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[3,:]/np.sum(Analitical_data[3:6,:],axis=0),"--g",linewidth=lf)
ax11.plot(time,xd[4,:]/np.sum(xd[3:6,:],axis=0),".r",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[4,:]/np.sum(Analitical_data[3:6,:],axis=0),"--r",linewidth=lf)
ax11.plot(time,xd[5,:]/np.sum(xd[3:6,:],axis=0),".b",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[5,:]/np.sum(Analitical_data[3:6,:],axis=0),"--b",linewidth=lf)

rect_1=[1.8,0,.7,.7]
ax1=fig.add_axes(rect_1,facecolor=(1,1,1))
ax1.plot(time,xd[6,:],".g",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[6,:],"--g",linewidth=lf)
ax1.plot(time,xd[7,:],".r",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[7,:],"--r",linewidth=lf)
ax1.plot(time,xd[8,:],".b",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[8,:],"--b",linewidth=lf)
plt.xlim(0,546)
plt.xlabel(r"$Date$",fontsize=xlf+1)
labels=["Jan 2021","Apr 2021","Jul 2021","Oct 2021","Jan 2022","Apr 2022","Jul 2022"]
plt.xticks([0,90,181,272,365,455,546], labels,fontsize=13,rotation=45 )
#plt.grid(linestyle='--', linewidth=2)
tit=ageg[2]
plt.title(tit,fontsize=titf)
plt.ylabel("Frequency",fontsize=xlf)
ax11=ax1.twinx()
ax11.plot(time,xd[6,:]/np.sum(xd[6:,:],axis=0),".g",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[6,:]/np.sum(Analitical_data[6:,:],axis=0),"--g",linewidth=lf)
ax11.plot(time,xd[7,:]/np.sum(xd[6:,:],axis=0),".r",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[7,:]/np.sum(Analitical_data[6:,:],axis=0),"--r",linewidth=lf)
ax11.plot(time,xd[8,:]/np.sum(xd[6:,:],axis=0),".b",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[8,:]/np.sum(Analitical_data[6:,:],axis=0),"--b",linewidth=lf)


plt.savefig("age_fit_republic.pdf", bbox_inches='tight', dpi=500)




In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
import pandas as pd
df3=pd.read_csv("paras-agemod2.csv")
data_0=np.zeros([9])
xd=np.zeros([9,536])
i=0
s=0
zd=np.zeros([3,5])
statelist=df3["state"]
for state in statelist:
    if df3["politic"].iloc[i]=="D":
        print(state)
        df = pd.read_csv(f'age-weekly/{state}.csv')
        df2=df.drop(columns="Dates")
        dataa=df2.to_numpy().T
        xd+=dataa/np.sum(dataa,axis=0)
        s+=1
    i+=1
    df = pd.read_csv(f'age-weekly/'+state+'.csv')
    time=np.arange(0, len(df['Dates']), 1)
data_0=list(xd[:,0]/s)
xd=xd/s
for i in range(9):
    xd[i,:]=replace_nan_with_average(xd[i,:])
data_0=list(np.average(xd[:,0:8],axis=1))
Bounds=((0,1),(0,1),(0,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(-1,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))
result = differential_evolution(loss, bounds=Bounds,args=(data_0, time,xd),strategy='best1bin',maxiter=1000, popsize=25,tol=1e-7,  atol=1e-7, mutation=(0.5, 1), recombination=0.7,   disp=False,polish=True )
best_params = result.x
print(best_params)
Analitical_data=(odeint(model, data_0, time, args=tuple(best_params))).T 
np.savetxt("age-Republic-model.txt",Analitical_data)
np.savetxt("age-Republic-survey.txt",xd)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Analitical_data=np.loadtxt("age-democratic-model.txt")
xd=np.loadtxt("age-democratic-model.txt")
mf=5
lf=2.7
titf=21
xlf=21
mf=5
lf=2.7
titf=21
xlf=21
ageg=["Age 18-34","Age 35-64", "Age above 64"]
fig=plt.figure(figsize=(10,7))
rect_1=[0,0,.7,.7]
ax1=fig.add_axes(rect_1,facecolor=(1,1,1))
ax1.plot(time,xd[0,:],".g",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[0,:],"--g",linewidth=lf,label="Accepted")
ax1.plot(time,xd[1,:],".r",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[1,:],"--r",linewidth=lf,label="Refused")
ax1.plot(time,xd[2,:],".b",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[2,:],"--b",linewidth=lf,label="Vaccinated")
plt.legend(fontsize=20,loc=2)
#plt.ylim(-0.01,0.5)
plt.xlim(0,546)
plt.xlabel(r"$Date$",fontsize=xlf+1)
labels=["Jan 2021","Apr 2021","Jul 2021","Oct 2021","Jan 2022","Apr 2022","Jul 2022"]
plt.xticks([0,90,181,272,365,455,546], labels,fontsize=13,rotation=45 )
#labels=["0.0","0.1","0.2","0.3","0.4","0.5"]
#plt.yticks([0,0.1,.2,0.3,.4,0.5], labels,fontsize=13 )
tit=ageg[0]
plt.title(tit,fontsize=titf)
plt.ylabel("Frequency",fontsize=xlf)
ax11=ax1.twinx()
ax11.plot(time,xd[0,:]/np.sum(xd[0:3,:],axis=0),".g",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[0,:]/np.sum(Analitical_data[0:3,:],axis=0),"--g",linewidth=lf)
ax11.plot(time,xd[1,:]/np.sum(xd[0:3,:],axis=0),".r",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[1,:]/np.sum(Analitical_data[0:3,:],axis=0),"--r",linewidth=lf)
ax11.plot(time,xd[2,:]/np.sum(xd[0:3,:],axis=0),".b",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[2,:]/np.sum(Analitical_data[0:3,:],axis=0),"--b",linewidth=lf)


rect_1=[0.9,0,.7,.7]
ax1=fig.add_axes(rect_1,facecolor=(1,1,1))
ax1.plot(time,xd[3,:],".g",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[3,:],"--g",linewidth=lf)
ax1.plot(time,xd[4,:],".r",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[4,:],"--r",linewidth=lf)
ax1.plot(time,xd[5,:],".b",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[5,:],"--b",linewidth=lf)

plt.xlim(0,546)
plt.xlabel(r"$Date$",fontsize=xlf+1)
labels=["Jan 2021","Apr 2021","Jul 2021","Oct 2021","Jan 2022","Apr 2022","Jul 2022"]
plt.xticks([0,90,181,272,365,455,546], labels,fontsize=13,rotation=45 )
tit=ageg[1]
plt.title(tit,fontsize=titf)
plt.ylabel("Frequency",fontsize=xlf)

#labels=[""]
#plt.yticks([0.0,0.2,0.4,0.6,0.8,1.0], labels,fontsize=13 )
ax11=ax1.twinx()
ax11.plot(time,xd[3,:]/np.sum(xd[3:6,:],axis=0),".g",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[3,:]/np.sum(Analitical_data[3:6,:],axis=0),"--g",linewidth=lf)
ax11.plot(time,xd[4,:]/np.sum(xd[3:6,:],axis=0),".r",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[4,:]/np.sum(Analitical_data[3:6,:],axis=0),"--r",linewidth=lf)
ax11.plot(time,xd[5,:]/np.sum(xd[3:6,:],axis=0),".b",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[5,:]/np.sum(Analitical_data[3:6,:],axis=0),"--b",linewidth=lf)

rect_1=[1.8,0,.7,.7]
ax1=fig.add_axes(rect_1,facecolor=(1,1,1))
ax1.plot(time,xd[6,:],".g",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[6,:],"--g",linewidth=lf)
ax1.plot(time,xd[7,:],".r",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[7,:],"--r",linewidth=lf)
ax1.plot(time,xd[8,:],".b",markersize=mf,alpha=0.5)
ax1.plot(time,Analitical_data[8,:],"--b",linewidth=lf)
plt.xlim(0,546)
plt.xlabel(r"$Date$",fontsize=xlf+1)
labels=["Jan 2021","Apr 2021","Jul 2021","Oct 2021","Jan 2022","Apr 2022","Jul 2022"]
plt.xticks([0,90,181,272,365,455,546], labels,fontsize=13,rotation=45 )
#plt.grid(linestyle='--', linewidth=2)
tit=ageg[2]
plt.title(tit,fontsize=titf)
plt.ylabel("Frequency",fontsize=xlf)
ax11=ax1.twinx()
ax11.plot(time,xd[6,:]/np.sum(xd[6:,:],axis=0),".g",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[6,:]/np.sum(Analitical_data[6:,:],axis=0),"--g",linewidth=lf)
ax11.plot(time,xd[7,:]/np.sum(xd[6:,:],axis=0),".r",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[7,:]/np.sum(Analitical_data[6:,:],axis=0),"--r",linewidth=lf)
ax11.plot(time,xd[8,:]/np.sum(xd[6:,:],axis=0),".b",markersize=mf,alpha=0.5)
ax11.plot(time,Analitical_data[8,:]/np.sum(Analitical_data[6:,:],axis=0),"--b",linewidth=lf)


plt.savefig("age_fit_democratic.pdf", bbox_inches='tight', dpi=500)


